In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install lion-pytorch

In [4]:
%pip install torchmetrics

In [5]:
import torch

from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from lion_pytorch import Lion
from utility.step_lr import StepLR


In [6]:

def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.1)
        loss.mean().backward()

        optimizer.step()
        optimizer.zero_grad()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

In [7]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [8]:
if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'widerestnet-lion'
    model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = WideResNet(16, 8, dropout=0.0, in_channels=3, labels=2).to(device)

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=32)
    optimizer = Lion(model.parameters(), lr=0.00002, weight_decay=0.5)#learning rate nie moze byc mniejszy, bo sie nie uczy
    scheduler = StepLR(optimizer, 0.00001, num_epochs)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()
    # torch.save(model.state_dict(), '../model_trained'+model_name+'.pth')

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.3669  │      0.57 %  ┃   1.000e-05  │       00:10  ┃      0.3677  │     50.00 %  ┃
┃           1  ┃      0.3622  │      0.68 %  ┃   1.000e-05  │       00:02  ┃      0.3655  │     42.33 %  ┃
┃           2  ┃      0.3529  │      0.82 %  ┃   1.000e-05  │       00:02  ┃      0.3594  │     62.67 %  ┃
┃           3  ┃      0.3410  │      0.83 %  ┃   1.000e-05  │       00:02  ┃      0.3475  │     74.00 %  ┃
┃           4  ┃      0.3273  │      0.85 %  ┃   1.000e-05  │       00:02  ┃      0.3306  │     76.00 %  ┃
┃           5  ┃      0.3134  │      0.86 %  ┃   1.000e-05  │       00:02  ┃      0.3158  │     76.33 %  ┃
┃           6  ┃      0.3011  │      0.88 %  ┃   1.000e-05  │       00:02  ┃      0.3092  │     78.67 %  ┃
┃           7  ┃      0.2895  │      0.88 %  ┃   1.000e-05  │       00:02  ┃      0.3059  │     78.00 %  ┃
┃           8  ┃      0.2771  │      0.88 %  ┃   1.000e-05  │       00:02  ┃      0.3037  │     77.33 %  ┃
┃           9  ┃      0.2661  │      

RuntimeError: ignored

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9709, device='cuda:0')
tensor(0.9700, device='cuda:0')
tensor(0.9434, device='cuda:0')
tensor(0.9400, device='cuda:0')
tensor(1., device='cuda:0')
